# Dataloader
TODO:
 - multiple time steps

In [2]:
%load_ext autoreload
%autoreload 2
%cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [3]:
from src.data import DataModule
import matplotlib.pyplot as plt
import numpy as np
from src.configuration import Config

In [1]:
data = DataModule(Config())
data.setup('fit')
loader = data.train_dataloader()
batch = next(iter(loader))

NameError: name 'DataModule' is not defined

In [34]:
print(f"input shape:  {batch[0]['input'].shape}, type: {batch[0]['input'].type()}, device: {batch[0]['input'].device}")
print(f"target shape: {batch[0]['target'].shape}, type: {batch[0]['target'].type()}, device: {batch[0]['target'].device}")

input shape:  torch.Size([64, 2, 72, 72]), type: torch.FloatTensor, device: cpu
target shape: torch.Size([64, 2, 72, 72]), type: torch.FloatTensor, device: cpu


In [75]:
import time
start_time = time.time()
for i, batch in enumerate(loader):
    input, target = batch[0]['input'], batch[0]['target']
print(f"epoch loading time: {(time.time() - start_time): 3.1f} seconds")

epoch loading time:  41.0 seconds


In [78]:
len(loader)

353

In [ ]:
'''
DALI original loading time ~47.5s

'''

In [3]:
data = DataModule(Config())
input_dataset = data.get_dataset('train', data.input_fname, data.input_variables)
target_dataset = data.get_dataset('train', data.target_fname, data.target_variables)

In [28]:
from src.data import ExternalInputCallable, DaliLoader

input_source = ExternalInputCallable(input_dataset,
                                     data.input_variables,
                                     data.train_batch_size,
                                     int(len(target_dataset.time)),
                                     get_indices(int(len(target_dataset.time))),
                                     time_axis=target_dataset.time
                                     )

target_source = ExternalInputCallable(target_dataset,
                                      data.target_variables,
                                      data.train_batch_size,
                                      int(len(target_dataset.time)),
                                      get_indices(int(len(target_dataset.time))),
                                      )

In [ ]:
from 

In [26]:
def get_indices( num_samples, shuffle=True):
    import random
    if shuffle:
        indices = random.sample(range(num_samples), num_samples)
    else:
        indices = list(np.arange(num_samples))
    return indices

In [9]:
from src.dataloader import DaskDataset
from torch.utils.data import DataLoader

data = DataModule(Config())
input_dataset = data.get_dataset('train', data.input_fname, data.input_variables)
target_dataset = data.get_dataset('train', data.target_fname, data.target_variables)

In [10]:
dataset = DaskDataset('train',
                      target_dataset,
                      input_dataset,
                      ['precipitation', 'temperature'],
                      ['precipitation', 'temperature'] )

In [34]:
dataloader = DataLoader(dataset,
                        batch_size=32,
                        shuffle=True,
                        drop_last=True, 
                        prefetch_factor=2,
                        num_workers=4)

In [35]:
import torch
torch.cuda.is_available()
torch.ones(12).to('cuda')

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')

In [36]:
import time
start_time = time.time()
for i, batch in enumerate(dataloader):
    input, target = batch[0], batch[1]
print(f"epoch loading time: {(time.time() - start_time): 3.1f} seconds")

epoch loading time:  14.2 seconds


In [67]:
batch[0][:,1].min(), batch[0][:,1].max()

(tensor(-0.6749), tensor(0.8714))

In [60]:
batch[1][:,1].min(), batch[1][:,1].max()

(tensor(213.0773), tensor(311.7216))

In [61]:
batch[1][:,0].min(), batch[1][:,0].max()

(tensor(0.), tensor(0.0016))

In [79]:
len(dataloader)

354

In [4]:
from src.dataloader import PyTorchDataModule

In [5]:
data = PyTorchDataModule(Config())

In [6]:
data.setup('fit')
train_loader = data.train_dataloader()
valid_loader = data.valid_dataloader()

In [7]:

batch = next(iter(train_loader))
batch = next(iter(valid_loader))

In [9]:
batch[0].shape


torch.Size([64, 2, 72, 144])